In [1]:
import pandas as pd
import numpy as np
import itertools
from itertools import groupby
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import random
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import torch
import os
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
pd.options.mode.chained_assignment = None 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
random_seed_list=np.load('F:/manuscript_2022_figure/model_scenario/20_ensemble_withoutAGL/same_para/rs_info_bi.npy'
)
fig_path='C:/Users/swei46.ASURITE/OneDrive/Documents/weekly_repo/manuscript_irri_timing/_figs4/'

import classification_daily_SPINUP_Bidirection as clss
import pros_cls_dat_0120_variables_SPINUP_BIdirection as pdc
import shapely.wkt
import captum
from captum.attr import IntegratedGradients, Occlusion, LayerGradCam, LayerAttribution

In [2]:
index_well=pd.read_excel('D:/index_well_KS_NE_TX/NE_TX_KS_well_location_xlsx.xlsx')
well_condition=index_well.iloc[:,-1]
index_well_name=index_well[well_condition.isnull()]['well_no']
columns_well=index_well_name.tolist()
# fold_pth='D:/NE_TX_KS_time_series/ready_state_01-20/'
fold_pth='D:/NE_TX_KS_time_series/ready_state_0120_v4/'
time_var=pd.read_csv('D:/NE_TX_KS_time_series/time_indicator_15May_15Oct.csv', index_col=0)
time_var.index=pd.to_datetime(time_var.index)

In [3]:
"'configurations'"
# index_well=pd.read_excel('D:/index_well_KS_NE_TX/NE_TX_KS_well_location_xlsx.xlsx')
# index_well_name=index_well.iloc[:,0]
variables=[ 
    'ld_blue', 'ld_green', 'ld_nir', 'ld_red','ld_swir1', 'ld_swir2', 'ld_thermal1', #'ld_thermal2',       
        'mod_blue','mod_et', 'mod_green', 'mod_nir', 'mod_red','mod_swir1', 'mod_swir2'  
    ,'noah_mp',
       'tem', 'vpd',
    'doy','pr_5mmthrehold'
    ,'denisarea'        #'agri_area',   
]
check_c=['well_level','time','irrilabel','name_col','event_label','pr_label']#,'irrigation_RMSE',

variables_withy=variables+check_c#'irrigation_RMSE',

time_indicator=['doy']#['year_decimal','month_numerical','June','July',	'August','September']


columns_well=index_well_name.tolist()
# columns_well=['TX_46','TX_43']

print(columns_well)

n_step=122
no_class_y=2
split_fraction=0.2
split_fraction_2=0.2
fil_nan=20
start_dat='2008-01-01'
end_dat='2020-12-31'
start_year=2008
end_year=2020

['NE_4', 'NE_6', 'NE_14', 'NE_18', 'NE_21', 'NE_24', 'NE_50', 'NE_53', 'NE_57', 'KS_1', 'KS_4', 'KS_6', 'KS_11', 'KS_12', 'KS_13', 'KS_15', 'KS_20', 'KS_21', 'KS_22', 'KS_23', 'KS_26']


In [4]:
ready_data=pdc.make_sequence(columns_well,fold_pth,time_indicator,start_year, end_year,variables_withy,time_var)
data_X,data_y,data_W=pdc.ready_data(ready_data,variables, check_c)

In [5]:
data_X1=torch.load('F:/manuscript_2022_figure/model_scenario/indexwell_with_gridscale_input/data_input_indexwell_grid.pt')
data_y=torch.load('F:/manuscript_2022_figure/model_scenario/indexwell_with_gridscale_input/data_target_indexwell_grid.pt')
data_W=np.load('F:/manuscript_2022_figure/model_scenario/indexwell_with_gridscale_input/data_info_indexwell_grid.npy',allow_pickle=True)

In [7]:
len(data_W)

134

In [3]:
data_X = np.delete(data_X1, -3, axis=-1)

In [4]:
rs=15
random_seed=rs
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
X_trcv, X_te,y_trcv,y_te, W_trcv,W_te=train_test_split(data_X,data_y,data_W,test_size=0.2, random_state=random_seed)
X_tr, X_cv,y_tr,y_cv, W_tr,W_cv=train_test_split(X_trcv,y_trcv,W_trcv,test_size=0.2, random_state=random_seed)
tensor_x_te= torch.from_numpy(np.asarray(X_te).astype(np.float32))
tensor_y_te=torch.tensor(np.asarray(y_te),dtype=torch.int64)

In [5]:
para_dict = {
  "n_features": data_X.shape[2],
  "seq_length": data_X.shape[1],
  "hidden_size_1": 64,
  "nb_layers_1":1,
  "drop_out_1":0,
  "drop_out_2":0,
  "learning_rate":5e-5,#1e-4,
    "train_episodes":800,
    "batch_size":16,
    "random_seed":0}

In [7]:
data_X.shape

torch.Size([134, 154, 19])

In [ ]:
all_mean= np.array([np.mean(data_X, axis=0).round() for i in range (data_X.shape[1]) ]
                 ,dtype=np.float32).reshape(1,data_X.shape[1],data_X.shape[2])

In [ ]:
# data_con=pd.concat(ready_data)
# data_con=data_X
# all_min=np.array([np.min(data_con[variables], axis=0).round() for i in range (data_X.shape[1]) ]
#                  ,dtype=np.float32).reshape(1,data_X.shape[1],data_X.shape[2])
# all_max= np.array([np.max(data_con[variables], axis=0).round() for i in range (data_X.shape[1]) ]
#                  ,dtype=np.float32).reshape(1,data_X.shape[1],data_X.shape[2])
all_mean= np.array([np.mean(data_con[variables], axis=0).round() for i in range (data_X.shape[1]) ]
                 ,dtype=np.float32).reshape(1,data_X.shape[1],data_X.shape[2])
bs_array = np.zeros((tensor_x_te.shape[0], data_X.shape[1], data_X.shape[2]), dtype=np.float32) 

# bs_array[:,:,variables.index('denisarea')]=all_min[:,:,variables.index('denisarea')]#irrigated area set as min
# bs_array[:,:,variables.index('doy')]=all_min[:,:,variables.index('doy')]#doy set as min
# bs_array[:,:,variables.index('mod_et')]=all_mean[:,:,variables.index('mod_et')]#irrigated area set as min
# bs_array[:,:,variables.index('pr_5mmthrehold')]=all_mean[:,:,variables.index('pr_5mmthrehold')]#doy set as min
# bs_array[:,:,variables.index('agri_area')]=all_mean[:,:,variables.index('agri_area')]#doy set as min

# bs_array[:,:,variables.index('vpd')]=all_min[:,:,variables.index('vpd')]#irrigated area set as min
# bs_array[:,:,variables.index('noah_mp')]=all_max[:,:,variables.index('noah_mp')]#doy set as min
baseline_ig= torch.tensor(bs_array ).to(device)

In [ ]:
step_ig=[]
step_count=[]
retrieve_dat=[]
date_index=np.arange (0,data_X.shape[1])+1
for rs_ind in range(20):
    model_load =clss.MV_LSTM(para_dict['n_features'],para_dict['seq_length'], para_dict['hidden_size_1'], 
                    para_dict['nb_layers_1'], para_dict['drop_out_1'],para_dict['drop_out_2'])
    model_load_path='F:/manuscript_2022_figure/model_scenario/20_ensemble_withoutAGL/same_para/model'+str(rs_ind)+'.pt'
    model_load.load_state_dict(torch.load(model_load_path))
    model_load.to(device)
    
    model_int=model_load.train()

    ig = IntegratedGradients(model_int,multiply_by_inputs=True)

    for k_sequence in range (17,139):
        target_output =(k_sequence,1)
        attributions = ig.attribute(tensor_x_te.to(device), target=target_output,baselines=baseline_ig)#,baselines=baseline_ig
        attr_numpy=attributions.cpu().data.numpy()#.reshape(tensor_x_te.size(1)
                                                       #,tensor_x_te.size(2))
        step_before_current=k_sequence-date_index
        step_ig.append(attr_numpy)
        step_count.append(step_before_current)

In [ ]:
saveig=np.array(step_ig,dtype=np.float32)
np.save('F:/manuscript_2022_figure/model_scenario/20_ensemble_withoutAGL/same_para/ig_baseline0.npy'
        , saveig)
savestep=np.array(step_count,dtype=np.float32)
np.save('F:/manuscript_2022_figure/model_scenario/20_ensemble_withoutAGL/same_para/step_baseline0.npy'
        , savestep)

# Varaibles atribution

In [ ]:
step_ig=np.load('F:/manuscript_2022_figure/model_scenario/20_ensemble_withoutAGL/same_para/ig_baseline0.npy')
step_count=np.load('F:/manuscript_2022_figure/model_scenario/20_ensemble_withoutAGL/same_para/step_baseline0.npy')

In [ ]:
variables_name=[ 
    'LANDSAT Blue', 'LANDSAT Green', 'LANDSAT NIR', 'LANDSAT Red','LANDSAT SWIR1', 'LANDSAT SWIR2', 'LANDSAT Thermal', 
        'MODIS Blue','MODIS ET', 'MODIS Green', 'MODIS NIR', 'MODIS Red','MODIS SWIR1', 'MODIS SWIR2'  
    ,'Noah-MP Soil moisture',
       'Temperature', 'VPD',
    'DOY','Precipitation'
    ,'Irrigated land percentage'           #'Agricultural land percentage',
]

In [ ]:
ig_a=np.array(step_ig)
prlabel_ready=np.array(W_te)[:,:,-1]#
pr_mask=np.nonzero(prlabel_ready)
ig_a[:,pr_mask[0],pr_mask[1],8]=0
ig_b=ig_a.mean(axis=0).mean(axis=0).mean(axis=0)

In [ ]:
ig_a.shape

In [ ]:
ig_top=ig_a[:,[24,0,29,25,40],:,:]
ig_top_mean=ig_top.mean(axis=0).mean(axis=0).mean(axis=0)

In [ ]:
ig_top.shape

In [ ]:
state_info=np.array(W_te)[:,:,-3][:,0]
ne_mask=["NE" in na for na in state_info]
ks_mask=["KS" in na for na in state_info]
ig_ne=ig_a[:,ne_mask,:,:].mean(axis=0).mean(axis=0).mean(axis=0)
ig_ks=ig_a[:,ks_mask,:,:].mean(axis=0).mean(axis=0).mean(axis=0)
ig_select=ig_top.reshape(20,122,5,154,20)#[:,:,:,:].mean(axis=0).mean(axis=0).mean(axis=0)
ig_july=ig_select[:,30:61,:,:,:].mean(axis=0).mean(axis=0).mean(axis=0).mean(axis=0)
ig_aug=ig_select[:,61:92,:,:,:].mean(axis=0).mean(axis=0).mean(axis=0).mean(axis=0)

In [ ]:
ig_select[:,61:92,:,:,:].shape

In [ ]:
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Times']})
# rc.rcParams['font.size'] = 12
rc('text', usetex=True)
txt_size=15
label_size=13

In [ ]:
# attri_frame.to_csv('F:/manuscript_2022_figure/model_scenario/20_ensemble_run/models_and_other_outputs/feature_acttribution_0.csv')

In [ ]:
attri_frame=pd.DataFrame(data=list(zip(ig_top_mean,ig_july,ig_aug)), index=variables_name, columns=['attributon','july','aug']).sort_values(by=['attributon'],axis=0)


In [ ]:
fig, ax1=plt.subplots(nrows=1, ncols=1, sharex= True, figsize=(10,5), dpi=200)
ax1.bar(np.arange(0,attri_frame.shape[0]), attri_frame['attributon'])
ax1.set_xticks(ticks=np.arange(0,attri_frame.shape[0]),labels=attri_frame.index, rotation=90)
ax1.tick_params(direction='in',labelsize=label_size)
ax1.set_ylabel('Integrated gradient',fontsize=txt_size)
plt.grid()
fig.tight_layout() 
# fig.savefig(fig_path+'feature_attribution.tif',bbox_inches='tight',dpi=500)    

plt.show()

## KS VS NE all feature

In [ ]:
txt_size=15
label_size=14
fig, axe=plt.subplots(nrows=2, ncols=1, sharex= True, sharey= True, figsize=(12,6), dpi=200)
ax=axe.flatten()
plot_x2=np.arange(0,attri_frame.shape[0])
ax[1].bar(plot_x2-0.15, attri_frame['july'],0.3,label='July',color='b',alpha=0.3)

ax[1].bar(plot_x2+0.15, attri_frame['aug'],0.3,label='August',color='orange',alpha=0.3)
ax[1].set_xticks(ticks=np.arange(0,attri_frame.shape[0]),labels=attri_frame.index, rotation=90)
ax[1].tick_params(direction='in',labelsize=label_size)
ax[1].set_ylabel('Integrated gradient',fontsize=txt_size)
ax[1].legend( loc='upper left',fontsize=txt_size, frameon=False)
# plt.grid()
# fig.tight_layout() 
# fig.savefig(fig_path+'feature_attribution.tif',bbox_inches='tight',dpi=500)    

# plt.show()

ax[0].bar(np.arange(0,attri_frame.shape[0]), attri_frame['attributon'],label='Averaged')
ax[0].set_xticks(ticks=plot_x2,labels=attri_frame.index, rotation=90)
ax[0].tick_params(direction='in',labelsize=label_size)
ax[0].xaxis.set_ticks_position('both')
ax[0].set_ylabel('Integrated gradient',fontsize=txt_size)
ax[0].legend( loc='upper left',fontsize=txt_size, frameon=False)

ax[1].grid()
ax[0].grid()
fig.tight_layout() 

fig.savefig(fig_path+'feature_attribution_ks_ne.tif',bbox_inches='tight',dpi=500)    



In [ ]:
fig, axe=plt.subplots(nrows=1, ncols=1, sharex= True, figsize=(20,8), dpi=500)

plot_x2=np.arange(0,attri_frame.shape[0])
axe.bar(plot_x2-0.15, attri_frame['NE'],0.3,label='Nebraska',color='b',alpha=0.3)

axe.bar(plot_x2+0.15, attri_frame['KS'],0.3,label='Kansas',color='orange',alpha=0.3)
axe.set_xticks(ticks=np.arange(0,attri_frame.shape[0]),labels=attri_frame.index, rotation=90)
axe.tick_params(direction='in',labelsize=label_size)
axe.set_ylabel('Integrated gradient',fontsize=txt_size)
axe.grid()
fig.tight_layout() 

# fig.savefig(fig_path+'step_arttribution.tif',bbox_inches='tight',dpi=500)    



# Step attribution


In [ ]:
ig_step=np.transpose(ig_select[:,30:92,:,:,:].reshape(-1,5,154,20), (1, 0, 2, 3))
step_jg=np.array(step_count).reshape(20,122,154)


In [ ]:
step_jg.shape

In [ ]:
step_arry=step_jg[:,30:92,:].reshape(-1,154)

In [ ]:
step_arry.shape

In [ ]:
## use 1 becasue python start from 0 , so the date_index also start from 0
# but where should we put the current day, as future or as past ?
mask = (step_arry >=0)&(step_arry <=15)
mask_2=(step_arry <1)&(step_arry >=-15)

In [ ]:
ig_back_all=ig_step[:,mask,:].mean(axis=1)#.mean(axis=0)
ig_future_all=ig_step[:,mask_2,:].mean(axis=1)#.mean(axis=0)

In [ ]:
ig_step[:,mask,:].shape,ig_step[:,mask_2,:].shape

In [ ]:
ig_back=ig_back_all.mean(axis=0)
ig_future=ig_future_all.mean(axis=0)

In [ ]:
ig_back_all.shape

In [ ]:
ig_step_all=pd.DataFrame(data=list(zip(ig_back,ig_future))
                         , index=variables_name, columns=['back','future'])

In [ ]:
step_var=[ 
    'Noah-MP Soil moisture','VPD',
       'Temperature', 
    'Precipitation'#, 'MODIS ET'#,'LANDSAT NIR','MODIS NIR'
]

In [ ]:
txt_size=12
label_size=10
fig, ax1=plt.subplots(nrows=1, ncols=1, sharex= True, figsize=(5,2), dpi=200)

plot_x=np.arange(0,len(step_var))
ax1.bar(plot_x-0.1, ig_step_all.loc[step_var]['back'],0.15,label='Backward',color='b',alpha=0.3)

ax1.bar(plot_x+0.1, ig_step_all.loc[step_var]['future'],0.15,label='Foward',color='r',alpha=0.3)

ax1.set_xticks(ticks=plot_x,labels=ig_step_all.loc[step_var].index)
ax1.tick_params(direction='in',labelsize=label_size)
ax1.legend( loc='upper right',fontsize=txt_size, frameon=False)#,bbox_to_anchor=(0.56, 1.05)

ax1.xaxis.set_ticks_position('both')
ax1.set_ylabel('Integratead gradient',fontsize=txt_size)
# ax1.set_ylim([-0.02, 0.15])
plt.grid()
fig.tight_layout() 
fig.savefig(fig_path+'step_arttribution.tif',bbox_inches='tight',dpi=500)    

plt.show()